# Purpose: 

The purpose of this notebook is to input the raw json files containing the amazon reviews and convert them into a tabular structure using pandas dataframes. 

In [1]:
import os 
import json
import pandas as pd 
import warnings
from tqdm import tqdm
# ignore all warnings
import warnings
warnings.filterwarnings("ignore")


In [2]:
os.getcwd()

'/home/ec2-user/SageMaker/amazon_thesis_detectron/000_Create_Manual_Annotation_Files'

In [3]:
# path to the raw json files containing amazon reviews: 
path = '/home/ec2-user/SageMaker/office_raw_data'
os.listdir(path)

['amazon_reviews_office_7.json',
 'amazon_reviews_office_4.json',
 'amazon_reviews_office_5.json',
 'amazon_reviews_office_0.json',
 'amazon_reviews_office_2.json',
 'amazon_reviews_office_8.json',
 'amazon_reviews_office_3.json',
 'amazon_reviews_office_1.json',
 'amazon_reviews_office_6.json',
 'amazon_reviews_office_9.json']

In [10]:
# path to save the new dataframe generated from this notebook 
path_to_saved_location = '/home/ec2-user/SageMaker/amazon_thesis_detectron/000_Create_Manual_Annotation_Files/001_office_data_processed'

In [11]:
def create_data_for_tagging(path_to_file,filename, path_to_saved_location): 
    """
    Read JSON data from a file, preprocess it, and save it as a DataFrame in JSON format.

    Args:
        path_to_file (str): Path to the directory containing the JSON file.
        filename (str): Name of the JSON file.
        path_to_saved_location (str): Path to the directory where the processed DataFrame will be saved.

    Returns:
        None
    """
    # Read data from the JSON file
    data = [json.loads(line) for line in open(os.path.join(path_to_file,filename),'r')]
    prod_master_list = []
    for product in data: 
        # print(product['asin'])
        if 'reviews' in product: 
            if product['reviews'] is  not None:
                prod_master_list.append(product)
        else:
            continue
    # incorporate product id and overall rating into every review body:
    for product in prod_master_list: 
        for review in product['reviews']: 
            review['product_id']= product['asin']
            review['overall_rating']= product['rating']
        # now remove only the reviews and append to a new list: 
    reviews = []
    for prod in prod_master_list: 
        reviews.append(prod['reviews'])
    master =[]
    for lst in tqdm(reviews):
        # loop through each dictionary in the list
        for dct in lst:
            # add the dictionary to the dataframe
            master.append(dct)
    # Create the DataFrame
    #         amazon_reviews_0_df = amazon_reviews_0_df.append(dct, ignore_index=True)
    amazon_reviews_0_df = pd.DataFrame(columns = ['review_id','account_name', 'account_link', 'review_date', 'rating', 'review_title', 'review_body'
                                      'purchase_type', 'helpful_votes', 'product_id', 'overall_rating'])
    amazon_reviews_0_df = amazon_reviews_0_df.append(master, ignore_index=True)
    # Save the DataFrame as JSON
    amazon_reviews_0_df.to_json(os.path.join(path_to_saved_location, f'{filename}.json'))
    return 'Sucess!'

In [12]:
for file in os.listdir(path): 
    create_data_for_tagging(path, file,path_to_saved_location )

100%|██████████| 2125/2125 [00:00<00:00, 124071.11it/s]


In [13]:
print("End of Notebook")

End of Notebook
